# 從每日的加密貨幣價格資料，找出當年度交易量最大的一天

> 1. 原始資料欄位有交易日（date）、開盤價（open）、收盤價（close）、最高價（max）、最低價（min）以及交易量（vol）
> 2. 為了要以年為單位操作，須先建立 year 欄位，接著後續使用 groupby 找出年度最大值

In [55]:
import pandas as pd

df = pd.read_csv('C:/Users/JEFF TU/Downloads/throwsh.csv')
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df['year'] = df['date'].dt.year

df.head(5)

,date,open,max,min,close,vol,year
0,1997-07-10,8.0476,8.0476,8.0476,8.0476,7190075,1997
1,1997-07-11,8.2190,8.2190,8.2190,8.2190,4547878,1997
2,1997-07-14,7.3994,7.3994,7.3994,7.3994,1162876,1997
3,1997-07-15,6.6816,6.6816,6.6816,6.6816,2430180,1997
4,1997-07-16,7.3298,7.3298,7.3298,7.3298,1666283,1997


# 方法一：join
> 以 year & max vol 做 key，用原 df join 回 date 資料

In [57]:
year_max = pd.DataFrame(df.groupby('year')['vol'].max()).reset_index(level=0) # reset_index 不然 year 會變成 index
pd.merge(year_max, df, on=['year', 'vol'], how='inner')[['date', 'vol']]

,date,vol
0,1997-07-10,7190075
1,1998-02-09,3968369
2,1999-07-07,6231927
3,2000-01-14,5259179
4,2001-10-11,7681942
5,2002-01-10,4323656
6,2003-08-27,6453973
7,2004-11-05,12269671
8,2005-02-18,9256851
9,2006-01-31,13121030


# 方法二：找出最大值後用 index 抓資料
> 缺點是若該年度有兩個日期具有相同的 vol，只會抓一筆出來

In [56]:
a1 = df.loc[df.groupby('year')['vol'].idxmax(), 'date'].tolist()
a2 = df.loc[df.groupby('year')['vol'].idxmax(), 'vol'].tolist()

pd.DataFrame(list(zip(a1, a2)), columns=['date', 'vol'])

,date,vol
0,1997-07-10,7190075
1,1998-02-09,3968369
2,1999-07-07,6231927
3,2000-01-14,5259179
4,2001-10-11,7681942
5,2002-01-10,4323656
6,2003-08-27,6453973
7,2004-11-05,12269671
8,2005-02-18,9256851
9,2006-01-31,13121030


# 方法三：用 transform 建出一個 vol_max 欄位
> 有了新欄位後，簡單使用 vol == vol_max 比對找出該年度交易量最大的資料，並透過 iloc 取出原資料，比起方法二，這個作法不會遺漏相同數值的資料

In [71]:
df['vol_max'] = df.groupby('year')['vol'].transform(max)
result = df.loc[df['vol'] == df['vol_max']]

result[['date', 'vol']].reset_index(drop=True)

,date,vol
0,1997-07-10,7190075
1,1998-02-09,3968369
2,1999-07-07,6231927
3,2000-01-14,5259179
4,2001-10-11,7681942
5,2002-01-10,4323656
6,2003-08-27,6453973
7,2004-11-05,12269671
8,2005-02-18,9256851
9,2006-01-31,13121030
